In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/Shareddrives/'directory'/

/content/drive/Shareddrives/Prof Lu - MUNI climate change/Fall 2022


In [ ]:
!pip install --pre --upgrade gensim
import pandas as pd
import gensim
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")
gensim.__version__
import numpy as np

import string
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

import pandas as pd
from nltk.stem.snowball import SnowballStemmer
import glob

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
def article_summary(docs_df_sentence):
  digit_pc_list = []
  for index in range(len(docs_df_sentence)):
    sent = docs_df_sentence.iloc[index]
    a_id = sent['article_id']
    for i,k in enumerate(sent['text_tokens']):
      alphanum_cnt = 0
      digit_chunk_cnt = 0
      for s in k:
        if re.match('[a-z]',s):
          alphanum_cnt += 1
          break
      for s in sent['text_tokens_digit'][i]:
        if s == '0':
          digit_chunk_cnt +=1

      if alphanum_cnt != 0 :
        temp_dict = dict()
        temp_dict['article_id'] = a_id
        temp_dict['sent_id'] = i
        temp_dict['sent_length'] = len(sent['text_tokens'][i])
        temp_dict['digit_chunk_ratio'] = digit_chunk_cnt/len(sent['text_tokens'][i])
        digit_pc_list.append(temp_dict)
  digit_pc_df = pd.DataFrame(digit_pc_list)
  return digit_pc_df


def preprocess_adapt_sent(docs_df_sentence, sent_colname):

    """
    Perform sentence and word tokenization for the raw text data

    Parameters:
    ----------
    docs_df : pandas dataframe that contains documents information and raw text data

    Returns:
    -------
    docs_df_sentence : pandas dataframe that contains documents information and word tokens for each sentence
    """
    tqdm.pandas()

    def sep_bullets(x):
        result = []
        for line in x:
            result += line.split("•")
        return result

    # Combine digits and replace chunk of digits with 0
    def combine_digit(x):
        x = re.sub("\d+([^A-Za-z0-9\s]\d+)+",'0',x)
        x = re.sub("\d+",'0',x)
        return x

    # Separate Bullets:
    docs_df_sentence["sent_tokens"] = docs_df_sentence[sent_colname].progress_map(sep_bullets)
    # [End] Part 1

    # convert text to lowercase
    docs_df_sentence["text_tokens"] = docs_df_sentence["sent_tokens"].progress_map(lambda x: [i.lower() for i in x])

    # [Added] Part 2: Cleaning Process Before Combining Chunks and Calculating Digits
    # remove all next line
    docs_df_sentence["text_tokens_helper"] = docs_df_sentence["text_tokens"].progress_map(lambda x: [re.sub("\s+", " ", i) for i in x])
    # [End] Part 2

    # remove special characters and numbers
    docs_df_sentence["text_tokens"] = docs_df_sentence["text_tokens"].progress_map(lambda x: [re.sub("[^a-z()]+", " ", i) for i in x])

    # [Added] Part 3: Cleaning Process and Combining Chunks
    # Combine chunk of digits
    docs_df_sentence["text_tokens_digit"] = docs_df_sentence["text_tokens_helper"].progress_map(lambda x: [combine_digit(i) for i in x])
    # [End] Part 3

    # word tokenize every sentence
    docs_df_sentence["text_tokens"] = docs_df_sentence["text_tokens"].progress_map(lambda x: [word_tokenize(i) for i in x])

### Reload Saved Adapt Tokens and Raw Sentence

In [ ]:
adapt_stemmed_sents = []
topic_sent_df = []
for i,file in enumerate(glob.glob("Muni Climate Change/Processed_Pickled_adapt_sentence_230706_CSV/A*")):
  print(i,file)
  topic_sent_df_list = pd.read_csv(file)
  topic_sent_df.append(topic_sent_df_list[['article_id','state','city_index',
                                           'city_name', 'year', 'docs_type',
                                           'docs_name','kept_adapt_sentence',
                                           'text_tokens_stemmed']])
  for k in range(len(topic_sent_df_list)):
    temp = topic_sent_df_list.iloc[k]
    adapt_stemmed_sents += eval(temp['text_tokens_stemmed'])

topic_sent_df = pd.concat(topic_sent_df)
topic_sent_df['kept_adapt_sentence'] = topic_sent_df['kept_adapt_sentence'].apply(lambda x:eval(x))
topic_sent_df['text_tokens_stemmed'] = topic_sent_df['text_tokens_stemmed'].apply(lambda x:eval(x))

0 Muni Climate Change/Processed_Pickled_adapt_sentence_230706_CSV/AL_pickled_adapt_sentence.csv
1 Muni Climate Change/Processed_Pickled_adapt_sentence_230706_CSV/AR_pickled_adapt_sentence.csv
2 Muni Climate Change/Processed_Pickled_adapt_sentence_230706_CSV/AZ_pickled_adapt_sentence.csv
3 Muni Climate Change/Processed_Pickled_adapt_sentence_230706_CSV/CA_pickled_adapt_sentence.csv
4 Muni Climate Change/Processed_Pickled_adapt_sentence_230706_CSV/CO_pickled_adapt_sentence.csv
5 Muni Climate Change/Processed_Pickled_adapt_sentence_230706_CSV/CT_pickled_adapt_sentence.csv
6 Muni Climate Change/Processed_Pickled_adapt_sentence_230706_CSV/DE_pickled_adapt_sentence.csv
7 Muni Climate Change/Processed_Pickled_adapt_sentence_230706_CSV/DC_pickled_adapt_sentence.csv
8 Muni Climate Change/Processed_Pickled_adapt_sentence_230706_CSV/FL_pickled_adapt_sentence.csv
9 Muni Climate Change/Processed_Pickled_adapt_sentence_230706_CSV/GA_pickled_adapt_sentence.csv
10 Muni Climate Change/Processed_Pickled

## Topic Modeling

In [ ]:
from gensim import corpora
from gensim.models import ldamodel

In [ ]:
def preprocess_adapt_tokens(sent):
  stopwd = stopwords.words ('english')
  punc = string.punctuation
  new_sent = [[word for word in s \
          if word not in stopwd and word not in punc] for s in sent]
  return new_sent

def keywords_stemmer(keywords):

    """
    Stemming the keywords, if a keyword is not a unigram, stemming each unigram separately and then combine them together

    Parameters:
    ----------
    keywords : pandas dataframe of all the keywords

    Returns:
    -------
    keywords_stem : pandas dataframe of all the keywords with column keywords_stem added
    """

    #the stemmer requires a language parameter
    snow_stemmer = SnowballStemmer(language = "english")

    #stem's of each word
    stem_words = []

    for i in range(len(keywords)):

        if keywords.iloc[i]["Stemmed"] == 1:

            w = keywords.iloc[i]["Keyword"]
            unigram = []
            split_w = w.split()


            for s in split_w:
                x = snow_stemmer.stem(s)
                unigram.append(x)

            unigram = " ".join(unigram)
            stem_words.append(unigram)

        else:

            stem_words.append(keywords.iloc[i]["Keyword"])

    keywords["keywords_stem"] = stem_words
    keywords = keywords.fillna("")

    return keywords

def remove_keywords(sent, keywords):
  new_keywords = set()
  for k in keywords:
    for w in k.split(" "):
      new_keywords.add(w)
  return [[word for word in s \
          if word not in new_keywords] for s in sent]

def remove_words(sent, remove_words):
  """Remove single alphabet and those in remove_words """
  from collections import defaultdict
  remove_words_cnt = defaultdict(int)
  result = []
  for s in sent:
    temp = []
    for word in s:
      if len(word) ==1 or word in remove_words:
        remove_words_cnt[word] +=1
      else:
        temp.append(word)
    result.append(temp)
  return result,remove_words_cnt

def topic_model(target_details, num_topics, topicModel='lda'):
  """topicModel = ['lda', 'hdp']"""
  warnings.filterwarnings(action='once')
  dictionary = corpora.Dictionary(target_details)
  corpus = [dictionary.doc2bow(text) for text in target_details]
  if topicModel== 'lda':
    model = ldamodel.LdaModel(corpus, num_topics = num_topics,
                              id2word=dictionary, passes=15,
                              random_state = 100)
    topics = model.print_topics(num_words=10)
  for topic in topics:
      print(topic)
  return model, topics, dictionary


In [ ]:
import pandas as pd
import zipfile
from nltk.stem.snowball import SnowballStemmer
import glob
at_least_line = 2
keywords_adapt = pd.read_excel("Muni Climate Change/keywords_adaptation.xlsx")
keywords_adapt_stem = keywords_stemmer(keywords_adapt)

#### Clean data and train model

In [ ]:
# remove some common sentences
adapt_stemmed_sents = set()
patterns = ['Actual Actual Budget Budget'.lower(),
            'Capital Assets Capital assets, which include'.lower(),
            'It is responsible for the operation'.lower(),
            "MATA MSCAA MZS".lower(),
            "Statement of Net Position".lower(),
            "The following is a schedule of interfund transfers".lower(),
            "The remainder of the page is intentionally left blank".lower(),
            'BUDGET PREPARATION WORKSHEET'.lower(),
            'Proposed Expenditures Staffing City Department'.lower()]

def any_match(x,patterns):
  x = x.lower()
  for p in patterns:
    if p in x:
      return True
  return False
irrelevant_adapt_sent = set()
for k in range(len(topic_sent_df)):
  temp = topic_sent_df.iloc[k]
  for i,sent in enumerate(temp['kept_adapt_sentence']):
    if any_match(sent,patterns):
      irrelevant_adapt_sent.add(sent)
    else:
      adapt_stemmed_sents.add(" ".join(temp['text_tokens_stemmed'][i]))
adapt_stemmed_sents=[c.split(" ") for c in adapt_stemmed_sents]

In [ ]:
# remove some irregular words
import time
start_time = time.time()
adapt_sent_processed = preprocess_adapt_tokens(adapt_stemmed_sents)
adapt_sent_noKeywords = remove_keywords(adapt_sent_processed,
                                        keywords_adapt_stem['keywords_stem'].tolist())
adapt_sent_noIrrWords,irrWords_cnt = remove_words(adapt_sent_noKeywords,['project','total','fund','fy','year','citi'])
print((time.time()-start_time)/60)

0.24668237765630086


In [ ]:
# Run Topic Analysis
start_time = time.time()
tx_topic_model_dict = dict()
model, topics, dictionary = topic_model(adapt_sent_noIrrWords,5)
tx_topic_model_dict['lda_noIrrWords_noPatterns']={'model': model,'topics':topics,'dictionary':dictionary}
print((time.time()-start_time)/60)

(0, '0.032*"park" + 0.013*"includ" + 0.011*"sidewalk" + 0.011*"new" + 0.011*"phase" + 0.011*"st" + 0.010*"avenu" + 0.010*"th" + 0.008*"creek" + 0.008*"upgrad"')
(1, '0.014*"includ" + 0.013*"provid" + 0.010*"sewer" + 0.010*"maintain" + 0.008*"area" + 0.008*"complet" + 0.008*"develop" + 0.008*"increas" + 0.007*"public" + 0.007*"work"')
(2, '0.032*"bond" + 0.023*"revenu" + 0.020*"capit" + 0.019*"tax" + 0.018*"general" + 0.017*"debt" + 0.013*"million" + 0.010*"sewer" + 0.009*"use" + 0.009*"oblig"')
(3, '0.029*"public" + 0.017*"depart" + 0.016*"develop" + 0.016*"park" + 0.016*"work" + 0.014*"engin" + 0.011*"util" + 0.011*"fire" + 0.011*"communiti" + 0.010*"budget"')
(4, '0.060*"budget" + 0.036*"capit" + 0.025*"district" + 0.022*"cost" + 0.021*"descript" + 0.019*"adopt" + 0.017*"sourc" + 0.013*"expenditur" + 0.013*"cip" + 0.012*"impact"')
3.458766504128774


### Save Topic Results

In [ ]:
def topic_extract(df, model_name, topic_model_dict):
  if len(df['text_tokens_stemmed'])==0:
    return None
  text_tokens_stemmed_comb = [c for sent in df['text_tokens_stemmed'] for c in sent]
  model_dict = topic_model_dict[model_name]
  temp =model_dict['dictionary'].doc2bow(text_tokens_stemmed_comb)
  result = []
  for k,v in sorted(model_dict['model'][temp], key = lambda x: -x[1]):
    result.append((k,round(v,2)))
  return result

def find_topic_pc(x,i):
  if type(x) != list:
    return None
  for k,v in x:
    if k == i:
      return v

In [ ]:
topic_sent_df['topic'] = topic_sent_df.apply(topic_extract,
                                                   args = ('lda_noIrrWords_noPatterns',tx_topic_model_dict,),
                                                   axis = 1)
for i in range(5):
  topic_sent_df[f'topic_{i}'] = topic_sent_df['topic'].apply(find_topic_pc, args = (i,))

<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()


In [ ]:
topics_df = []
for name, item in tx_topic_model_dict.items():
  for topic_i, topic in item['topics']:
    temp = dict()
    temp['model'] = name
    temp['model_path'] = f'Topic Modeling/topic_models/{name}_topic_modeling_v1.gensim'
    item['model'].save(f'Topic Modeling/topic_models/{name}_topic_modeling_v1.gensim')
    item['dictionary'].save(f'Topic Modeling/topic_models/{name}_topic_modeling_v1.dictionary')

    temp['topic_index'] = topic_i
    temp['topic'] = topic

    topics_df.append(temp)
topics_df = pd.DataFrame(topics_df)
topics_df.to_csv("Topic Modeling/model_output(topic_definition).csv",index = False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Reload Topic Modeling Models

In [ ]:
def topic_extract(df, model, dictionary):
  if len(df['text_tokens_stemmed'])==0:
    return None
  text_tokens_stemmed_comb = [c for sent in df['text_tokens_stemmed'] for c in sent]
  temp =dictionary.doc2bow(text_tokens_stemmed_comb)
  result = []
  for k,v in sorted(model[temp], key = lambda x: -x[1]):
    result.append((k,round(v,2)))
  return result

def find_topic_pc(x,i):
  if type(x) != list:
    return None
  for k,v in x:
    if k == i:
      return v

In [ ]:
name = 'lda_noIrrWords_noPatterns'
model = ldamodel.LdaModel.load(f'Topic Modeling/topic_models/{name}_topic_modeling_v1.gensim')
dictionary = corpora.Dictionary.load(f'Topic Modeling/topic_models/{name}_topic_modeling_v1.dictionary')

In [ ]:
topic_sent_df['topic'] = topic_sent_df.apply(topic_extract,
                                                   args = (model,dictionary),
                                                   axis = 1)
for i in range(5):
  topic_sent_df[f'topic_{i}'] = topic_sent_df['topic'].apply(find_topic_pc, args = (i,))

In [ ]:
topic_sent_ind_df = []
for i in range(len(topic_sent_df)):
  temp = topic_sent_df.iloc[i]
  for raw_sent,stemmed_sent in zip(temp['kept_adapt_sentence'],temp['text_tokens_stemmed']):
    temp_dict = dict()

    for k in topic_sent_df.columns:
      if k not in ['kept_adapt_sentence','text_tokens_stemmed']:
        temp_dict[k] = temp[k]
    temp_dict['kept_adapt_sentence'] = raw_sent
    temp_dict['text_tokens_stemmed'] = stemmed_sent
    temp_dict['topic'] = model[dictionary.doc2bow(stemmed_sent)]
    topic_sent_ind_df.append(temp_dict)
topic_sent_ind_df = pd.DataFrame(topic_sent_ind_df)

In [ ]:
topic_sent_ind_df = pd.DataFrame(topic_sent_ind_df)

In [ ]:
for i in range(5):
  topic_sent_ind_df[f'topic_{i}'] = topic_sent_ind_df['topic'].apply(find_topic_pc, args = (i,))

In [ ]:
topic_sent_ind_df.to_csv("Topic Modeling/final_output_topics.csv", index = False)